<a href="https://colab.research.google.com/github/poltorashka-s-BMa/course-os-linux/blob/main/E_Papers_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задача F: Классификация научных статей по цитированиям
Добро пожаловать на соревнование!

В этом ноутбуке мы разберем задачу классификации научных статей, подготовим данные, создадим простой бейзлайн и основной шаблон для построения графовой нейронной сети, а также сформируем файл с решением для отправки в систему.

### Легенда задачи

Вы работаете в научном издательстве, которое управляет огромной базой данных статей. Каждая статья относится к одной из 20 научных областей (например, "биоинформатика", "машинное обучение", "квантовая физика"). Ваша задача — восстановить утерянные метки областей для 440 статей, используя текстовые эмбеддинги и граф цитирований.

## Шаг 1: Загрузка и подготовка данных
Сначала импортируем все необходимые библиотеки и загрузим наши данные. Для этого соревнования мы сгенерируем данные прямо в ноутбуке, чтобы он был полностью самодостаточным.

In [ ]:
# Эти библиотеки вам наверняка пригодятся
import json
import random
import numpy as np
from collections import defaultdict, Counter

# Эти будут полезны для создания моделей
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, GATConv

# А эти могут понядобиться для расчета и визуализации результата
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [ ]:
# Загрузка данных
with open('embeddings_data.json', 'r') as f:
    embeddings_data = json.load(f)
with open('graph_data.json', 'r') as f:
    graph_data = json.load(f)

# Посмотрим на структуру
print("Пример элемента из embeddings_data (только ключи):")
print(embeddings_data[0].keys())
print("\nПример ребра из graph_data:")
print(graph_data[0])

# Разделим данные на обучающую и тестовую выборки
train_data = [d for d in embeddings_data if d['split'] == 'train']
test_data = [d for d in embeddings_data if d['split'] == 'test']

print(f"\nВсего вершин: {len(embeddings_data)}")
print(f"Из них в обучении (train): {len(train_data)}")
print(f"Из них в тесте (test): {len(test_data)}")
print(f"Всего ребер в графе: {len(graph_data)}")

Пример элемента из embeddings_data (только ключи):
dict_keys(['id', 'label', 'embedding', 'split'])

Пример ребра из graph_data:
[0, 280]

Всего вершин: 1440
Из них в обучении (train): 1000
Из них в тесте (test): 440
Всего ребер в графе: 11471


## Шаг 2: Бейзлайн на основе структуры графа
Давайте построим простое базовое решение (бейзлайн), которое использует только структуру графа и полностью игнорирует эмбеддинги. Логика простая: для каждой тестовой вершины предскажем тот класс, который чаще всего встречается среди её соседей из обучающей выборки.

Ваша задача — заполнить пропущенные части в коде ниже.

In [ ]:
def graph_baseline_predictor(graph_data, embeddings_data, num_classes=20):
    """
    Предсказывает классы для тестовых вершин на основе самого популярного класса
    среди соседей из обучающей выборки.
    """
    adj_list = defaultdict(list)
    for u, v in graph_data:
        adj_list[u].append(v)
        adj_list[v].append(u)  # Делаем граф двунаправленным

    node_info = {d['id']: d for d in embeddings_data}
    test_node_ids = [d['id'] for d in embeddings_data if d.get('split') == 'test']

    predictions = {}

    for node_id in test_node_ids:
        all_neighbors = adj_list.get(node_id, [])
        train_neighbors = [n for n in all_neighbors if n in node_info and node_info[n]['split'] == 'train']

        if not train_neighbors:
            predictions[node_id] = random.randint(0, num_classes - 1)
            continue

        neighbor_labels = [node_info[n_id]['label'] for n_id in train_neighbors]

        if not neighbor_labels:
            predictions[node_id] = random.randint(0, num_classes - 1)
            continue

        label_counts = Counter(neighbor_labels)
        predicted_class = label_counts.most_common(1)[0][0]
        predictions[node_id] = predicted_class

    return predictions


In [ ]:
baseline_predictions = graph_baseline_predictor(graph_data, embeddings_data)

**Подсказка:** Если вы все сделаете правильно, точность этого бейзлайна должна быть в районе 0.4. По формуле оценки это даст вам около 2 баллов. Ваша цель — значительно превзойти этот результат!

### Шаг 3: Функция для сохранения решения
Определим функцию для выгрузки решения сразу. Она понадобится нам и для бейзлайна, и для финальной модели.

In [ ]:
def save_predictions_to_csv(predictions_dict, filename="predictions.csv"):
    """
    Сохраняет предсказания из словаря в CSV файл для отправки в систему.
    """
    # Убедимся, что predictions_dict содержит предсказания для всех тестовых вершин
    test_ids = {d['id'] for d in embeddings_data if d['split'] == 'test'}
    if set(predictions_dict.keys()) != test_ids:
        print("Внимание! В словаре предсказаний отсутствуют некоторые тестовые ID.")

    pred_df = pd.DataFrame(list(predictions_dict.items()), columns=['id', 'label'])
    pred_df = pred_df.sort_values(by='id')
    pred_df.to_csv(filename, header=False, index=False)

    print(f"\nРешение успешно сохранено в файл: {filename}")
    print("Пример содержимого файла:")
    # Магическая команда для вывода первых строк файла в Jupyter
    !head -n 5 {filename}

In [ ]:
save_predictions_to_csv(baseline_predictions, "baseline_predictions.csv")


Решение успешно сохранено в файл: baseline_predictions.csv
Пример содержимого файла:
1000,18
1001,4
1002,3
1003,8
1004,10


## Шаг 4: Построение финальной модели (GNN)
Теперь ваша очередь построить графовую нейросеть! Вам предстоит пройти все этапы: подготовка данных, определение архитектуры, обучение и получение предсказаний.

### 4.1: Подготовка данных и создание выборок
Сначала подготовим данные в формате PyTorch Geometric. Самое важное — мы разделим обучающую выборку (train) на две части: новую, уменьшенную обучающую (new_train) и валидационную (val).

Валидационная выборка критически важна для:

- Настройки гиперпараметров (скорость обучения, глубина сети и т.д.).
- Отслеживания переобучения и применения Early Stopping.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# --- Создание тензоров ---
all_labels = {d['id']: d.get('label', -1) for d in embeddings_data}
x = torch.tensor([d['embedding'] for d in embeddings_data], dtype=torch.float)
y = torch.tensor([all_labels[i] for i in range(len(embeddings_data))], dtype=torch.long)
edge_index = torch.tensor(graph_data, dtype=torch.long).t().contiguous()

# --- Создание масок ---
# Сначала найдем ID всех обучающих вершин
train_ids = [d['id'] for d in embeddings_data if d['split'] == 'train']


train_ids_new, val_ids = train_test_split(
    train_ids,
    test_size=0.2,
    random_state=42,
    stratify=[all_labels[i] for i in train_ids]
)

# Теперь создаем маски PyTorch на основе этих ID
train_mask = torch.zeros(len(embeddings_data), dtype=torch.bool)
val_mask = torch.zeros(len(embeddings_data), dtype=torch.bool)
test_mask = torch.zeros(len(embeddings_data), dtype=torch.bool)

train_mask[train_ids_new] = True
val_mask[val_ids] = True
test_mask[[d['id'] for d in embeddings_data if d['split'] == 'test']] = True

# Собираем все в один объект Data
pyg_data = Data(x=x, edge_index=edge_index, y=y,
                train_mask=train_mask, val_mask=val_mask, test_mask=test_mask)

print("Объект данных PyTorch Geometric:")
print(pyg_data)
print(f"Новый размер train: {pyg_data.train_mask.sum().item()}")
print(f"Размер validation: {pyg_data.val_mask.sum().item()}")
print(f"Размер test: {pyg_data.test_mask.sum().item()}")

Объект данных PyTorch Geometric:
Data(x=[1440, 1024], edge_index=[2, 11471], y=[1440], train_mask=[1440], val_mask=[1440], test_mask=[1440])
Новый размер train: 800
Размер validation: 200
Размер test: 440


### 4.2: Определение архитектуры модели
Ваша задача — спроектировать и написать класс для GNN модели.

Примерный план архитектуры:

1. Вход: Принимает объект pyg_data.
2. Слой 1: Графовый сверточный слой (например, GCNConv или GATConv), который преобразует входные эмбеддинги в промежуточное представление. После него — функция активации (например, ReLU).
3. Регуляризация: Слой Dropout для борьбы с переобучением.
4. Слой 2: Еще один графовый сверточный слой.
5. Классификатор (MLP): После графовых слоев можно добавить небольшой полносвязный слой (nn.Linear) для финальной классификации.
6. Выход: Логиты для каждого из 20 классов.

In [ ]:
class MyGNN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout_p=0.6):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.bn1 = BatchNorm(hidden_channels)
        self.dropout1 = nn.Dropout(p=dropout_p)

        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = BatchNorm(hidden_channels)
        self.dropout2 = nn.Dropout(p=dropout_p)

        self.classifier_lin = nn.Linear(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.dropout1(x)

        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.dropout2(x)

        x = self.classifier_lin(x)
        return x

### 4.3: Обучение модели
Напишите цикл обучения. Следите за loss и accuracy на обучающей и валидационной выборках.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, BatchNorm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

in_channels = pyg_data.x.size(1)
hidden_channels = 128
out_channels = 20

model = MyGNN(in_channels, hidden_channels, out_channels, dropout_p=0.6).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)
scheduler = StepLR(optimizer, step_size=100, gamma=0.5)  # снижать lr каждые 100 эпох в 2 раза
criterion = nn.CrossEntropyLoss()

pyg_data = pyg_data.to(device)

best_val_acc = 0
best_model_state = None
patience = 30
counter = 0

for epoch in range(1, 501):
    model.train()
    optimizer.zero_grad()

    out = model(pyg_data)
    loss = criterion(out[pyg_data.train_mask], pyg_data.y[pyg_data.train_mask])
    loss.backward()
    optimizer.step()
    scheduler.step()

    model.eval()
    with torch.no_grad():
        out = model(pyg_data)
        pred = out.argmax(dim=1)

        train_acc = (pred[pyg_data.train_mask] == pyg_data.y[pyg_data.train_mask]).float().mean().item()
        val_acc = (pred[pyg_data.val_mask] == pyg_data.y[pyg_data.val_mask]).float().mean().item()

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_state = copy.deepcopy(model.state_dict())
        counter = 0
    else:
        counter += 1

    if epoch % 10 == 0 or epoch == 1:
        print(f"Эпоха: {epoch:03d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

    if counter >= patience:
        print(f"Ранняя остановка на эпохе {epoch}. Лучшая Val Acc: {best_val_acc:.4f}")
        break

model.load_state_dict(best_model_state)

if pyg_data.test_mask.sum() > 0 and (pyg_data.y[pyg_data.test_mask] >= 0).all():
    model.eval()
    with torch.no_grad():
        out = model(pyg_data)
        pred = out.argmax(dim=1)
        test_acc = (pred[pyg_data.test_mask] == pyg_data.y[pyg_data.test_mask]).float().mean().item()
    print(f"Точность на тесте: {test_acc:.4f}")
else:
    print("Нет меток для теста или тестовый набор пуст")


Эпоха: 001, Loss: 3.0613, Train Acc: 0.0637, Val Acc: 0.0650
Эпоха: 010, Loss: 1.7456, Train Acc: 0.1900, Val Acc: 0.1500
Эпоха: 020, Loss: 1.1251, Train Acc: 0.1688, Val Acc: 0.1600
Эпоха: 030, Loss: 0.8018, Train Acc: 0.2075, Val Acc: 0.1550
Эпоха: 040, Loss: 0.6229, Train Acc: 0.7625, Val Acc: 0.3500
Эпоха: 050, Loss: 0.4747, Train Acc: 0.7638, Val Acc: 0.3600
Эпоха: 060, Loss: 0.4286, Train Acc: 0.8425, Val Acc: 0.3650
Эпоха: 070, Loss: 0.3244, Train Acc: 0.9237, Val Acc: 0.3600
Эпоха: 080, Loss: 0.3009, Train Acc: 0.9100, Val Acc: 0.3550
Ранняя остановка на эпохе 83. Лучшая Val Acc: 0.4550
Нет меток для теста или тестовый набор пуст


## Шаг 5: Выгрузка решения
После того, как вы обучили вашу лучшую модель и получили словарь final_predictions, его нужно сохранить в predictions.csv в правильном формате для отправки в систему.

In [ ]:
# 1. Переключаем модель в eval режим
model.eval()

# 2. Предсказания для всех узлов
with torch.no_grad():
    out = model(pyg_data)
    preds = out.argmax(dim=1).cpu().numpy()  # Получаем классы

# 3. Выбираем только тестовые узлы
test_indices = pyg_data.test_mask.nonzero(as_tuple=False).view(-1).cpu().numpy()

# 4. Формируем словарь id: predicted_label для теста
final_predictions = {}
for idx in test_indices:
    article_id = idx.item()  # id вершины в графе
    predicted_label = int(preds[idx])
    final_predictions[article_id] = predicted_label

# Функция сохранения, если её нет, вот простая реализация:
def save_predictions_to_csv(predictions_dict, filename="predictions.csv"):
    with open(filename, "w") as f:
        for id_, label in sorted(predictions_dict.items()):
            f.write(f"{id_},{label}\n")

# Сохраняем в файл
save_predictions_to_csv(final_predictions)


Готово! Теперь у вас есть файл predictions.csv, который можно загружать в систему. Попробуйте улучшить свой результат!

In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.9 MB/s eta 0:00:00
